In [1]:
import pandas as pd

data=pd.read_csv('heart_disease_uci.csv')

In [2]:
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [3]:
# Assuming 'data' is a DataFrame or a structure like that
categorical_features = data.select_dtypes(include=['object']).columns.tolist()
numerical_features = data.select_dtypes(exclude=['object']).columns.tolist()
if 'id' in numerical_features:
    numerical_features.remove('id')
    numerical_features.remove('num')
    
print("categorical_features")
print(categorical_features)
print("\nnumerical_features")
print(numerical_features)

categorical_features
['sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']

numerical_features
['age', 'trestbps', 'chol', 'thalch', 'oldpeak', 'ca']


In [4]:
# replacing missing values with mean
from sklearn.impute import SimpleImputer

imputer=SimpleImputer(strategy='mean')

num_data=data.drop(categorical_features,axis=1)
imputer.fit(num_data)
imputer.statistics_




array([460.5       ,  53.51086957, 132.13240418, 199.13033708,
       137.54566474,   0.87878788,   0.6763754 ,   0.99565217])

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


In [6]:
# handling the missing values
data_imputed=imputer.transform(num_data)

imputed_df=pd.DataFrame(data_imputed,columns=num_data.columns)
data_1=pd.concat([imputed_df,data[categorical_features]],axis=1)

data_1.dropna(subset=categorical_features,inplace=True)  #inplace is used to ensure that new dataset is not created it just alter the original dataset
data_1.info()

# to check if rows are removed when categories value(s) are missing
# missing_categorical = data[categorical_features].isnull().sum()

# # Print or view the count of missing values in categorical columns
# print(missing_categorical)


# target = data_1.iloc[:, 7]  # Assuming the target column is at index 7

# # Remove the target column from the DataFrame to keep only features
# data_1 = data_1.drop(columns=data_1.columns[7])


# to ensure that the target and data have the same number of rows as some rows where removed in because of category
# len(data_1)
# len(target)



<class 'pandas.core.frame.DataFrame'>
Index: 372 entries, 0 to 903
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        372 non-null    float64
 1   age       372 non-null    float64
 2   trestbps  372 non-null    float64
 3   chol      372 non-null    float64
 4   thalch    372 non-null    float64
 5   oldpeak   372 non-null    float64
 6   ca        372 non-null    float64
 7   num       372 non-null    float64
 8   sex       372 non-null    object 
 9   dataset   372 non-null    object 
 10  cp        372 non-null    object 
 11  fbs       372 non-null    object 
 12  restecg   372 non-null    object 
 13  exang     372 non-null    object 
 14  slope     372 non-null    object 
 15  thal      372 non-null    object 
dtypes: float64(8), object(8)
memory usage: 49.4+ KB


In [7]:
# normalising

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
data_1[numerical_features]=scaler.fit_transform(data_1[numerical_features])
data_1.head()

,id,age,trestbps,chol,thalch,oldpeak,ca,num,sex,dataset,cp,fbs,restecg,exang,slope,thal
0,1.0,0.912088,0.719280,0.185953,0.247221,1.151950,-0.802174,0.0,Male,Cleveland,typical angina,True,lv hypertrophy,False,downsloping,fixed defect
1,2.0,1.355613,1.558087,0.730148,-1.364202,0.434753,2.773361,2.0,Male,Cleveland,asymptomatic,False,lv hypertrophy,True,flat,normal
2,3.0,1.355613,-0.678732,0.144881,-0.558491,1.420899,1.581516,1.0,Male,Cleveland,asymptomatic,False,lv hypertrophy,True,flat,reversable defect
3,4.0,-1.970826,-0.119527,0.360506,1.666808,2.227746,-0.802174,0.0,Male,Cleveland,non-anginal,False,normal,False,downsloping,normal
4,5.0,-1.527300,-0.119527,-0.111815,1.091300,0.345103,-0.802174,0.0,Female,Cleveland,atypical angina,False,lv hypertrophy,False,upsloping,normal


In [8]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 372 entries, 0 to 903
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        372 non-null    float64
 1   age       372 non-null    float64
 2   trestbps  372 non-null    float64
 3   chol      372 non-null    float64
 4   thalch    372 non-null    float64
 5   oldpeak   372 non-null    float64
 6   ca        372 non-null    float64
 7   num       372 non-null    float64
 8   sex       372 non-null    object 
 9   dataset   372 non-null    object 
 10  cp        372 non-null    object 
 11  fbs       372 non-null    object 
 12  restecg   372 non-null    object 
 13  exang     372 non-null    object 
 14  slope     372 non-null    object 
 15  thal      372 non-null    object 
dtypes: float64(8), object(8)
memory usage: 49.4+ KB


In [9]:
data_1['dataset'].value_counts()  #to see the categories in the car=tegorical_features
# 'sex', 'dataset', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal'


dataset
Cleveland        301
Switzerland       43
VA Long Beach     18
Hungary           10
Name: count, dtype: int64

In [15]:
binary_cat=['sex','fbs','exang']
nominal_cat=['dataset']
hot_categorical_features=binary_cat+nominal_cat
# selecting these from the data
hot_data=data_1[hot_categorical_features]

from sklearn.preprocessing import OneHotEncoder
hot_encoder=OneHotEncoder(sparse_output=False,drop='first')
encoded_hot_data=hot_encoder.fit_transform(hot_data)
encoded_hot_columns=hot_encoder.get_feature_names_out(hot_categorical_features)
encoded_hot_df=pd.DataFrame(encoded_hot_data,columns=encoded_hot_columns)



data_1.drop(hot_categorical_features, axis=1, inplace=True)  # Drop original categorical columns

# data_1 = pd.concat([data_1, encoded_hot_df], axis=1)






ordinal_cat=['cp','restecg','slope','thal']


KeyError: "None of [Index(['sex', 'fbs', 'exang', 'dataset'], dtype='object')] are in the [columns]"

In [17]:
duplicated_rows = data_1[encoded_hot_df.duplicated()]
print(duplicated_rows)

C:\Users\Aditya\AppData\Local\Temp\ipykernel_2116\1954193299.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  duplicated_rows = data_1[encoded_hot_df.duplicated()]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [11]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 372 entries, 0 to 903
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        372 non-null    float64
 1   age       372 non-null    float64
 2   trestbps  372 non-null    float64
 3   chol      372 non-null    float64
 4   thalch    372 non-null    float64
 5   oldpeak   372 non-null    float64
 6   ca        372 non-null    float64
 7   num       372 non-null    float64
 8   sex       372 non-null    object 
 9   dataset   372 non-null    object 
 10  cp        372 non-null    object 
 11  fbs       372 non-null    object 
 12  restecg   372 non-null    object 
 13  exang     372 non-null    object 
 14  slope     372 non-null    object 
 15  thal      372 non-null    object 
dtypes: float64(8), object(8)
memory usage: 49.4+ KB


In [104]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442 entries, 0 to 371
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     372 non-null    float64
 1   age                    372 non-null    float64
 2   trestbps               372 non-null    float64
 3   chol                   372 non-null    float64
 4   thalch                 372 non-null    float64
 5   oldpeak                372 non-null    float64
 6   ca                     372 non-null    float64
 7   num                    372 non-null    float64
 8   cp                     372 non-null    object 
 9   restecg                372 non-null    object 
 10  slope                  372 non-null    object 
 11  thal                   372 non-null    object 
 12  sex_Male               372 non-null    float64
 13  fbs_True               372 non-null    float64
 14  exang_True             372 non-null    float64
 15  dataset_Hun

In [106]:
duplicated_rows = data_1[data_1.duplicated()]
print(duplicated_rows)


     id  age  trestbps  chol  thalch  oldpeak  ca  num   cp restecg slope  \
304 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
305 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
307 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
308 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
310 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
313 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
314 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
315 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
316 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
317 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
318 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   
319 NaN  NaN       NaN   NaN     NaN      NaN NaN  NaN  NaN     NaN   NaN   

In [90]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'data_1' is your DataFrame containing ordinal categorical features

ordinal_cat = ['cp', 'restecg', 'slope', 'thal']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each ordinal categorical column
for feature in ordinal_cat:
    # Fit and transform the ordinal categorical feature
    data_1[feature] = label_encoder.fit_transform(data_1[feature])

In [91]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 442 entries, 0 to 371
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     372 non-null    float64
 1   age                    372 non-null    float64
 2   trestbps               372 non-null    float64
 3   chol                   372 non-null    float64
 4   thalch                 372 non-null    float64
 5   oldpeak                372 non-null    float64
 6   ca                     372 non-null    float64
 7   num                    372 non-null    float64
 8   cp                     442 non-null    int32  
 9   restecg                442 non-null    int32  
 10  slope                  442 non-null    int32  
 11  thal                   442 non-null    int32  
 12  sex_Male               372 non-null    float64
 13  fbs_True               372 non-null    float64
 14  exang_True             372 non-null    float64
 15  dataset_Hun

In [71]:
# target and data_1 find the correlation
corr_matrix=data_1.corr()
corr_matrix["num"].sort_values(ascending=False)

num                      1.000000
ca                       0.461479
oldpeak                  0.399445
thal                     0.269606
id                       0.246472
age                      0.227092
sex_Male                 0.172659
trestbps                 0.158367
exang_True               0.100930
restecg                  0.053343
fbs_True                -0.000151
dataset_Hungary         -0.028865
dataset_Switzerland     -0.043864
chol                    -0.239018
slope                   -0.353333
cp                      -0.424200
thalch                  -0.449275
dataset_VA Long Beach         NaN
Name: num, dtype: float64

In [72]:
# performing splitting

from sklearn.model_selection import train_test_split

# Assuming 'preprocessed_data' is your preprocessed dataset

# Splitting into training (70%) and temporary data (30%)
train_data, temp_data = train_test_split(data_1, test_size=0.3, random_state=42)

# Further splitting the temporary data into validation (20%) and test (10%)
validation_data, test_data = train_test_split(temp_data, test_size=1/3, random_state=42)

# Final ratios:
# Training: 70% | Validation: 20% | Test: 10%

# to verify the ratio
total_samples = len(data_1)
train_samples = len(train_data)
validation_samples = len(validation_data)
test_samples = len(test_data)

train_ratio = train_samples / total_samples
validation_ratio = validation_samples / total_samples
test_ratio = test_samples / total_samples

print(f"Training set ratio: {train_ratio:.2f}")
print(f"Validation set ratio: {validation_ratio:.2f}")
print(f"Test set ratio: {test_ratio:.2f}")

Training set ratio: 0.70
Validation set ratio: 0.20
Test set ratio: 0.10


In [73]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 309 entries, 226 to 103
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     266 non-null    float64
 1   age                    266 non-null    float64
 2   trestbps               266 non-null    float64
 3   chol                   266 non-null    float64
 4   thalch                 266 non-null    float64
 5   oldpeak                266 non-null    float64
 6   ca                     266 non-null    float64
 7   num                    266 non-null    float64
 8   cp                     309 non-null    int32  
 9   restecg                309 non-null    int32  
 10  slope                  309 non-null    int32  
 11  thal                   309 non-null    int32  
 12  sex_Male               256 non-null    float64
 13  fbs_True               256 non-null    float64
 14  exang_True             256 non-null    float64
 15  dataset_H

In [74]:
import numpy as np

class KNN:
    def __init__(self, k=3):
        self.k = k

    def euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = []
        for x_test in X:
            distances = [self.euclidean_distance(x_test, x_train) for x_train in self.X_train]
            # Sort by distance and return indices of the first k neighbors
            k_indices = np.argsort(distances)[:self.k]
            k_nearest_labels = [self.y_train[i] for i in k_indices]
            # Get the most common class label among the k neighbors
            most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
            predictions.append(most_common)
        return predictions


In [75]:
from sklearn.metrics import accuracy_score

train_label = train_data.iloc[:, 7]  # Assuming the target column is at index 7
test_label=test_data.iloc[:,7]
# Remove the target column from the DataFrame to keep only features
train_data = train_data.drop(columns=data_1.columns[7])
test_data=test_data.drop(columns=data_1.columns[7])

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)  # Set the number of neighbors (k)

# Fit the classifier to the training data
knn.fit(train_data, train_label)

# Make predictions on the test data
predictions = knn.predict(test_data)


# Calculate accuracy
accuracy = accuracy_score(test_label, predictions)
print(f"Accuracy of the KNN model: {accuracy:.2f}")


ValueError: Input X contains NaN.
KNeighborsClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values